In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import set_config

set_config(transform_output='pandas')


In [ ]:
class HousePriceClassifier:
    """
    Cassification pipeline for predicting whether a house is 'expensive' or 'not expensive'.
    Steps included:
        - preprocessing
        - encoding
        - model training
        - hyperparameter tuning.
    """

    def __init__(self, path):
        """
        initialize the classifier with data set
        Parameters:
        - path to csv file
        """

        self.data = pd.read_csv(path)
        self.ordinal_features = [
            'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
            'BsmtFinType1', 'KitchenQual', 'HeatingQC', 'GarageQual',
            'GarageCond', 'PoolQC', 'Fence'
        ]

        # order is: excellent, good, typical/average, fair, poor, (nor available)
        self.order_categories = [
            ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
            ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
            ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
            ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
            ['Gd', 'Av', 'Mn', 'No', 'NA'],
            ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
            ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
            ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
            ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
            ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
            ['Ex', 'Gd', 'TA', 'Fa', 'NA'],
            ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA']
        ]
        self.model = None
        self.search = None

    def prepare_data(self):
        """
        prepare training and test sets
        identify categorical and numerical columns
        """

        self.y = self.data.pop('Expensive')
        self.X = self.data.drop(columns=["Fireplaces"])
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=0.2, random_state=42
        )
        self.X_num = self.X_train.select_dtypes(include='number').columns
        self.X_cat = self.X_train.select_dtypes(exclude='number').columns

        self.oh_features = [col for col in self.X_cat if col not in self.ordinal_features]

    def build_preprocessor(self):
        """
        constructs the preprocessing pipeline for numerical and categorical data
        """

        # impute missing values with mean for numerical features and 'NA' for categorical features
        num_imputer = SimpleImputer(strategy='mean')
        cat_imputer = SimpleImputer(strategy='constant', fill_value='NA')

        # encode categorical data with ordinal encoder for features that have a 'logical' order
        # encode categorical data with one hot encoder for features with no order
        oh_encoder = OneHotEncoder(handle_unknown='infrequent_if_exist', sparse_output=False)
        ord_encoder = OrdinalEncoder(categories=self.order_categories)

        cat_encoder = ColumnTransformer(transformers=[
            ('oh', oh_encoder, self.oh_features),
            ('ord', ord_encoder, self.ordinal_features)
        ], verbose_feature_names_out=False)

        # full pipelines including both imputers and encoders
        self.num_pipeline = Pipeline([('imputer', num_imputer)])
        self.cat_pipeline = Pipeline([
            ('imputer', cat_imputer),
            ('encoder', cat_encoder)
        ])

        # final preprocessor
        self.preprocessor = ColumnTransformer(transformers=[
            ('num', self.num_pipeline, self.X_num),
            ('cat', self.cat_pipeline, self.X_cat)
        ], verbose_feature_names_out=False)

    def build_model(self):
        """
        build a complete RandomForest pipeline including preprocessing
        """
        self.model = Pipeline([
            ("preprocessor", self.preprocessor),
            ("classifier", RandomForestClassifier())
        ])

    def tune_model(self, n_iter=300, cv=5):
        """
        performs randomized hyperparameter search on the model
        """
        # ranges were found and refined after several tuning iterations
        param_grid = {
            'classifier__n_estimators': range(168, 175),
            'classifier__ccp_alpha': [10**e for e in range(-6, -4)],
            'classifier__max_depth': range(10, 14),
            'classifier__min_samples_split': range(2, 21, 2),
            'classifier__min_samples_leaf': range(1, 11)
        }

        self.search = RandomizedSearchCV(
            self.model,
            param_distributions=param_grid,
            n_iter=n_iter,
            cv=cv,
            verbose=1,
            n_jobs=-1,
            random_state=42
        )

        self.search.fit(self.X_train, self.y_train)

        print("Best Parameters:", self.search.best_params_)
        print("Best Cross-Validation Score:", self.search.best_score_)

    def evaluate(self):
        """Evaluates the tuned model on the test set."""
        best_model = self.search.best_estimator_
        y_pred = best_model.predict(self.X_test)
        acc = accuracy_score(self.y_test, y_pred)
        print("Test Accuracy:", acc)
        return acc
